# Fitting an early MLE
In this notebook we will fit an early MLE model to audiovisual speech perception data.

In [ ]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import binom
from scipy.optimize import minimize, LinearConstraint, Bounds

Error: Session cannot generate requests

In [ ]:
#https://github.com/seby-sbirna/Cognitive-Modeling/blob/master/W9%20-%20Bayesian%20Multisensory%20Integration/MLEexcersiseSolution.m

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [ ]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
data_A = data[:, 0, :]
data_V = data[:, 1, :]
data_AV = data[:, 2:, :]

## Fit MLE

In [ ]:


def compute_params(c_A, c_V, std_A, std_V):
    """Compute the parameters for the MLE model from the free parameters"""
    mu = np.arange(5) + 1
    den = 1 / (std_A + std_V) 
    mu_AV = ((mu - c_A) * std_V + (mu - c_V) * std_A) * den
    std_AV = std_A * std_V * den
    return mu_AV, std_AV

In [ ]:
compute_params(1, 1, 1, 1)